## 1) Given a direction of interest in the crystal lattice, how does it map on the EBSD detector?

#### Following https://www.sciencedirect.com/science/article/pii/S1044580316300924

#### To answer this we need a set of 3 variables:

#### 1) The angle $\alpha$  that maps the sample frame on the detector frame via an anticlockwise rotation around the x axis
#### $\alpha = \theta_{sample} - 90^{\circ} - \theta_{detector}$
#### $\theta_{sample}$ is the angle to the horizontal
#### $\theta_{detector}$ is the angle to the vertical

In [22]:
alpha = 30  #degrees

#### 2) The set of Euler angles in the Bunge convetion (ZXZ)

In [23]:
import sympy as sp
Euler = sp.Matrix([12., 14., 0.]) #degrees

#### 3) The crystal lattice basis vectors in the form of structure matrix

In [24]:
def a_hex(a, c):
    '''
    The direct space structure matrix of a hexagonal system
    define by lattice parameters a and c
    '''
    return sp.Matrix([[a, -a/2., 0.], [0., sp.sqrt(3.)*a/2., 0.], [0., 0., c]])

In [47]:
a_GaN = 3.14 #nm
c_GaN = 5.14 #nm

A_GaN = a_hex(a_GaN, c_GaN) #a, c in nm

### Let's say we want to know how the crystal's [001] maps on the detector

In [26]:
uvw_c = sp.Matrix([0, 0, 1]) # change this direction if you want a different one

#### No user modifications beyond this point
#### ---------------------------------------------------------------------------------------------------------

##### Right handed rotation matrix around z looks like this:

In [27]:
import mpmath as mp
def Rotation_z(gamma):
    return sp.Matrix([[sp.cos(mp.radians(gamma)), -sp.sin(mp.radians(gamma)), 0],\
                      [sp.sin(mp.radians(gamma)),  sp.cos(mp.radians(gamma)), 0],\
                      [ 0,                                                 0, 1]] )

##### Right handed rotation matrix around x looks like this:

In [28]:
def Rotation_x(gamma):
    return sp.Matrix([[1,                         0,                          0],\
                      [0, sp.cos(mp.radians(gamma)), -sp.sin(mp.radians(gamma))],\
                      [0, sp.sin(mp.radians(gamma)), sp.cos(mp.radians(gamma))]] )


###### A direction vector [uvw] defined in crystal frame will be in sample frame defined by the following components:

In [37]:
O = Rotation_z(Euler[2]) * Rotation_x(Euler[1]) * Rotation_z(Euler[0])

xyz_s = uvw_c.transpose() * A_GaN.transpose() * O

print xyz_s

Matrix([[0.258533726450217, 1.21630555377336, 4.98732003305862]])


##### In detector frame the vector [uvw]  will have components:

In [36]:
xyz_d = xyz_s * Rotation_x(alpha)

print xyz_d

Matrix([[0.0502983903599644, 0.690080063202556, 0.721983087265590]])





## 2) What is the normal to the sample's [001] in the crystal frame of the given grain?

In [40]:
xyz_s_known = sp.Matrix([0, 0, 1])

uvw_c = xyz_s_known.transpose() * O.transpose() * A_GaN.transpose().inv()

print uvw_c # These are just the vector coordinates in crystal frame not the Miller indices

Matrix([[-0.0444820610023353, -0.0889641220046705, 0.188773487602334]])


#### The direct space structure matrix for a kexagonal system is:

In [45]:
def g_hex(a, c):
    '''
    Real space metric tensor for hexagonal system
    define by lattice parameters a and c
    '''
    return sp.Matrix([[a*a,    -a*a/2.,  0.],\
                      [-a*a/2.,    a*a,  0.],\
                      [0.,          0., c*c]])

#### Then the vector we found will be in reciprocal space:


In [49]:
g_hkl = uvw_c *  g_hex(a_GaN, c_GaN)
print g_hkl

Matrix([[0, -0.657862992987937, 4.98732003305862]])


##### If these coordinates can be reduced to relative prime integers, ie Miller indices, then we can say we know the plane on which this direction is normal



